In [1]:
#Preferably this notebook should be run in Google Colab since Spark is required
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [93]:
#If wget does not work then simply download the dataset from the given URL
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

--2024-01-12 19:43:40--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip.1’

ml-latest-small.zip 100%[===================>] 955.28K   942KB/s    in 1.0s    

2024-01-12 19:43:42 (942 KB/s) - ‘ml-latest-small.zip.1’ saved [978202/978202]

Archive:  ml-latest-small.zip
replace ml-latest-small/links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
from sklearn.model_selection import train_test_split
df=pd.read_csv('ml-latest-small/ml-latest-small/ratings.csv')
df=df[['userId','movieId','rating']]
df.dropna()
train, test = train_test_split(df, test_size=0.1, random_state=13)


In [3]:
train.head(6)

,userId,movieId,rating
75433,477,76,4.0
6441,44,135,2.0
93474,599,2657,3.0
32995,225,1339,4.0
17109,109,165,3.0
13977,89,133867,4.5


In [4]:
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 316.9/316.9 MB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 200.5/200.5 kB ? eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425388 sha256=ab1db35633f390cc7128b4898877fda5bbeb12ec487ab40de19490e4a9c93988
  Stored in directory: c:\users\saras\appdata\local\pip\cache\wheels\72\3c\32\f0f20da5a933f8c6c5a1a2184a9e516652ed3eebeb49f5ddd0
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [ ]:
spark = SparkSession.builder.appName("app").getOrCreate()

# Assuming 'train' and 'test' are pandas DataFrames
# Convert them to PySpark DataFrames
train_spark = spark.createDataFrame(train)
test_spark = spark.createDataFrame(test)
als = ALS(
    maxIter=5,
    regParam=0.01,
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
)

# Fit the model on the training data
model = als.fit(train_spark)

In [99]:
predictions = model.transform(test_spark)
predictions.show(5)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   202|   2871|   4.0|  4.158884|
|   318| 134214|   4.0|  2.927055|
|    89| 109374|   3.5|  2.929971|
|   184|  87483|   2.0| 3.1740854|
|    77|   5816|   5.0| 7.0834417|
+------+-------+------+----------+
only showing top 5 rows



In [109]:
predictions_pd=predictions.toPandas()
predictions_pd.dropna(inplace=True)

In [110]:
predictions_pd[predictions_pd.isnull().any(axis=1)]

,userId,movieId,rating,prediction


In [111]:
actual_ratings = predictions_pd['rating']
predicted_ratings = predictions_pd['prediction']

# Calculate Mean Squared Error
mse = np.mean((actual_ratings - predicted_ratings) ** 2)

print(f"Mean Squared Error (MSE): {mse}")

Mean Squared Error (MSE): 1.098752188117083


In [113]:
movies=pd.read_csv('/content/ml-latest-small/movies.csv')
movies=movies.set_index('movieId')

In [150]:
def predict_movies(id_user):
  user_df = predictions_pd[predictions_pd['userId'] == id_user]
  top_10_movies = user_df.nlargest(10, 'prediction')[['movieId', 'prediction']]
  for x in list(top_10_movies['movieId']):
    print(movies.iloc[x]['title'])
  # print(top_10_movies['movieId'])

In [151]:
predict_movies(108)

Jack and Sarah (1995)
Teenage Mutant Ninja Turtles II: The Secret of the Ooze (1991)
Zeus and Roxanne (1997)
Germany Year Zero (Germania anno zero) (Deutschland im Jahre Null) (1948)
Hearts and Minds (1996)
King Solomon's Mines (1937)
Chaplin (1992)
Whatever It Takes (2000)
Pushing Tin (1999)
Nothing But Trouble (1991)
